In [166]:
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
import os
import json
import webbrowser
import pprint
import numpy as np

pp = pprint.PrettyPrinter(indent=2)

pd.set_option('display.max_columns', None)

In [2]:
files = os.listdir(os.getcwd())
print files

['.DS_Store', '.ipynb_checkpoints', '15_stanford_children.htm', '1_texas_children.htm', '2_boston_children.htm', '3_chicag_children.htm', '3_umich_children.htm', '5_wisconsin_children.htm', '6_cincinnati_children.htm', '8_la_children.htm', 'Best Hospitals for Pediatric Cardiology & Heart Surgery.htm', 'scrape_1.ipynb']


#### Overall Scores

In [3]:
url = 'Best Hospitals for Pediatric Cardiology & Heart Surgery.htm'


In [4]:
page = urllib.urlopen(url).read()
soup = BeautifulSoup(page, "lxml")

In [5]:
len(soup.find_all('script'))

17

In [6]:
script_data = [d for d in soup.find_all('script') if d.has_attr('id')]

script_data = [d for d in script_data if d.attrs['id'] == 'search-init-data']

data = script_data[0]

In [7]:
data = json.loads(data.text)

In [169]:
pre = 'https://health.usnews.com/'
for i in range(8):
    if i == 6: continue
    
    print data['matches'][i]['rank']
    print data['matches'][i]['name']
    for j in data['matches'][i]['scores']:
        print j
        
    suf = data['matches'][i]['full_scorecard_url']
    url2 = '%s%s' % (pre, suf)
    chrome_path = 'open -a /Applications/Google\ Chrome.app %s'
    if i < 2:
        webbrowser.get(chrome_path).open(url2)

1
Texas Children's Hospital
{u'score': u'100.0/100', u'title': u'Overall Score'}
{u'score': u'55.6%', u'title': u'Reputation Score'}
{u'score': u'10/12', u'title': u'Volume Score'}
2
Boston Children's Hospital
{u'score': u'97.3/100', u'title': u'Overall Score'}
{u'score': u'80.2%', u'title': u'Reputation Score'}
{u'score': u'12/12', u'title': u'Volume Score'}
3
Ann and Robert H. Lurie Children's Hospital of Chicago
{u'score': u'92.6/100', u'title': u'Overall Score'}
{u'score': u'14.9%', u'title': u'Reputation Score'}
{u'score': u'6/12', u'title': u'Volume Score'}
3
University of Michigan C.S. Mott Children's Hospital
{u'score': u'92.6/100', u'title': u'Overall Score'}
{u'score': u'46.1%', u'title': u'Reputation Score'}
{u'score': u'11/12', u'title': u'Volume Score'}
5
Children's Hospital of Wisconsin
{u'score': u'91.2/100', u'title': u'Overall Score'}
{u'score': u'16.5%', u'title': u'Reputation Score'}
{u'score': u'9/12', u'title': u'Volume Score'}
6
Cincinnati Children's Hospital Medi

#### Individual Scores

In [128]:
files = [f for f in os.listdir(os.getcwd()) if len(re.findall('^\d+_', f))==1]
files = [(int(f.split('_')[0]), f) for f in files]
files.sort()

print files

[(1, '1_texas_children.htm'), (2, '2_boston_children.htm'), (3, '3_chicag_children.htm'), (3, '3_umich_children.htm'), (5, '5_wisconsin_children.htm'), (6, '6_cincinnati_children.htm'), (8, '8_la_children.htm'), (15, '15_stanford_children.htm')]


In [22]:
url2 = files[0][1]
page2 = urllib.urlopen(url2).read()
soup2 = BeautifulSoup(page2, 'lxml')

In [45]:
texts = '\nUSN.ready().then(function() {\n    USN.EventDispatcher.instance.dispatcher.trigger(\'summon:hospitals:profile'
print texts


USN.ready().then(function() {
    USN.EventDispatcher.instance.dispatcher.trigger('summon:hospitals:profile


In [46]:
script_data = [d for d in soup2.find_all('script') if d.text.startswith(texts)]
print len(script_data)

1


In [103]:
d = json.loads(script_data[0].text.split('init(')[1].rstrip('\n});\n});\n')[:-7])
# print d['scorecard'].keys()
print d['scorecard']['measure_groups'][0]['measures'][0]['title']
print d['scorecard']['measure_groups'][0]['measures'][0]['score']['value']
print d['scorecard']['measure_groups'][0]['measures'][0]['score']['label']

Survival after congenital heart surgery
17/21
High


In [109]:
for group in d['scorecard']['measure_groups']:
    for measure in group['measures']:
        title = measure['title']
        score = measure['score']['value']
        label = measure['score']['label']
        print '%s: %s, %s' % (title, score, label)

Survival after congenital heart surgery: 17/21, High
Survival after certain complex heart procedures: 20/24, Very high
Survival after Norwood/hybrid surgery: 9/12, High
Survival after heart transplant: 5/6, High
Ability to prevent infections throughout hospital: 37/38, Excellent
Ability to prevent infections in intensive-care units: 3/5, Average
Ability to prevent pressure injuries: 5/5, Excellent
Number of surgeries: 10/12, High
Number of catheter procedures: 33/33, Very high
Number of Norwood or hybrid surgeries: 12/12, Very high
Nurse staffing: 3.7, Excellent
Congenital heart program: 23/23, Excellent
Adult congenital heart program: 10/10, Excellent
Heart transplant program: 11/11, Excellent
Advanced clinical services offered: 16/16, Excellent
Clinical support services offered: 9/9, Excellent
Advanced technologies available: 8/8, Excellent
Specialized clinics and programs available: 13/13, Excellent
Has fulltime subspecialists available: 19/20, Excellent
Recognized as Nurse Magnet h

#### Tabulate all hospitals

In [182]:
cols = ['Rank', 'Hospital', 'Measure', 'Score', 'Label']

In [183]:
df = pd.DataFrame(columns=cols)

In [184]:
files = [f for f in os.listdir(os.getcwd()) if len(re.findall('^\d+_', f))==1]
files = [(int(f.split('_')[0]), f) for f in files]
files.sort()

print files

[(1, '1_texas_children.htm'), (2, '2_boston_children.htm'), (3, '3_chicag_children.htm'), (3, '3_umich_children.htm'), (5, '5_wisconsin_children.htm'), (6, '6_cincinnati_children.htm'), (8, '8_la_children.htm'), (15, '15_stanford_children.htm')]


In [185]:
for tup in files:
    url2 = tup[1]
    rank = int(url2.split('_')[0])
    hospital = url2.split('_')[1]
    print rank, hospital
    page2 = urllib.urlopen(url2).read()
    soup2 = BeautifulSoup(page2, 'lxml')
    script_data = [d for d in soup2.find_all('script') if d.text.startswith(texts)]
    d = json.loads(script_data[0].text.split('init(')[1].rstrip('\n});\n});\n')[:-7])
    all_titles = []
    for group in d['scorecard']['measure_groups']:
        for measure in group['measures']:
            title = measure['title']
            score = measure['score']['value']
            label = measure['score']['label']
            data_list = [rank, hospital, title, score, label]
            df2 = pd.DataFrame(columns=cols, data=[data_list])
            df = df.append(df2)
            all_titles.append(title)

1 texas
2 boston
3 chicag
3 umich
5 wisconsin
6 cincinnati
8 la
15 stanford


In [186]:
display(df.Hospital.value_counts())

texas         28
stanford      28
umich         28
boston        28
la            28
cincinnati    28
wisconsin     28
chicag        28
Name: Hospital, dtype: int64

In [187]:
display(all_titles)
print len(set(all_titles))

[u'Survival after congenital heart surgery',
 u'Survival after certain complex heart procedures',
 u'Survival after Norwood/hybrid surgery',
 u'Survival after heart transplant',
 u'Ability to prevent infections throughout hospital',
 u'Ability to prevent infections in intensive-care units',
 u'Ability to prevent pressure injuries',
 u'Number of surgeries',
 u'Number of catheter procedures',
 u'Number of Norwood or hybrid surgeries',
 u'Nurse staffing',
 u'Congenital heart program',
 u'Adult congenital heart program',
 u'Heart transplant program',
 u'Advanced clinical services offered',
 u'Clinical support services offered',
 u'Advanced technologies available',
 u'Specialized clinics and programs available',
 u'Has fulltime subspecialists available',
 u'Recognized as Nurse Magnet hospital',
 u'Reputation with physicians in specialty',
 u'Commitment to best practices',
 u'Commitment to quality improvement',
 u'Adoption of health information technology',
 u'Active fellowship program',
 u'

28


In [188]:
display(df.head(3))

,Rank,Hospital,Measure,Score,Label
0,1,texas,Survival after congenital heart surgery,17/21,High
0,1,texas,Survival after certain complex heart procedures,20/24,Very high
0,1,texas,Survival after Norwood/hybrid surgery,9/12,High


In [189]:
df2 = df.pivot(index='Hospital', columns='Measure', values='Score')
df2 = df2[all_titles]
display(df2)

Measure,Survival after congenital heart surgery,Survival after certain complex heart procedures,Survival after Norwood/hybrid surgery,Survival after heart transplant,Ability to prevent infections throughout hospital,Ability to prevent infections in intensive-care units,Ability to prevent pressure injuries,Number of surgeries,Number of catheter procedures,Number of Norwood or hybrid surgeries,Nurse staffing,Congenital heart program,Adult congenital heart program,Heart transplant program,Advanced clinical services offered,Clinical support services offered,Advanced technologies available,Specialized clinics and programs available,Has fulltime subspecialists available,Recognized as Nurse Magnet hospital,Reputation with physicians in specialty,Commitment to best practices,Commitment to quality improvement,Adoption of health information technology,Active fellowship program,Commitment to clinical research,Help for families,Enlists families in structuring care
Hospital,,,,,,,,,,,,,,,,,,,,,,,,,,,,
boston,14/21,22/24,9/12,5/6,37/38,1/5,5/5,12/12,33/33,12/12,4.5,23/23,10/10,11/11,16/16,9/9,8/8,13/13,20/20,True,80.2%,50/50,14/15,16/16,7/7,12/12,8/8,7/7
chicag,18/21,23/24,11/12,5/6,38/38,2/5,5/5,6/12,18/33,5/12,3.2,23/23,10/10,11/11,16/16,9/9,7/8,13/13,18/20,True,14.9%,50/50,14/15,14/16,7/7,12/12,8/8,7/7
cincinnati,15/21,21/24,10/12,5/6,36/38,1/5,4/5,8/12,28/33,11/12,4.4,23/23,10/10,10/11,16/16,9/9,8/8,13/13,19/20,True,31.4%,49/50,15/15,16/16,7/7,12/12,8/8,7/7
la,14/21,16/24,12/12,6/6,36/38,3/5,4/5,10/12,29/33,12/12,3.3,23/23,10/10,10/11,16/16,9/9,7/8,13/13,19/20,True,18.9%,49/50,14/15,16/16,7/7,12/12,8/8,7/7
stanford,12/21,22/24,9/12,4/6,36/38,1/5,3/5,11/12,32/33,11/12,3.9,23/23,10/10,11/11,16/16,9/9,8/8,13/13,19/20,False,47.3%,47/50,14/15,16/16,7/7,12/12,8/8,7/7
texas,17/21,20/24,9/12,5/6,37/38,3/5,5/5,10/12,33/33,12/12,3.7,23/23,10/10,11/11,16/16,9/9,8/8,13/13,19/20,True,55.6%,48/50,15/15,15/16,7/7,12/12,8/8,7/7
umich,15/21,19/24,10/12,6/6,32/38,4/5,3/5,11/12,28/33,12/12,3.7,23/23,10/10,10/11,15/16,9/9,7/8,13/13,19/20,True,46.1%,46/50,15/15,14/16,7/7,12/12,8/8,7/7
wisconsin,18/21,16/24,12/12,3/6,30/38,5/5,5/5,9/12,19/33,11/12,4.5,23/23,10/10,11/11,16/16,9/9,8/8,13/13,20/20,True,16.5%,50/50,12/15,16/16,6/7,12/12,8/8,7/7


In [190]:
df3 = df.pivot(index='Hospital', columns='Measure', values='Label')
df3 = df3[all_titles]
display(df3)

Measure,Survival after congenital heart surgery,Survival after certain complex heart procedures,Survival after Norwood/hybrid surgery,Survival after heart transplant,Ability to prevent infections throughout hospital,Ability to prevent infections in intensive-care units,Ability to prevent pressure injuries,Number of surgeries,Number of catheter procedures,Number of Norwood or hybrid surgeries,Nurse staffing,Congenital heart program,Adult congenital heart program,Heart transplant program,Advanced clinical services offered,Clinical support services offered,Advanced technologies available,Specialized clinics and programs available,Has fulltime subspecialists available,Recognized as Nurse Magnet hospital,Reputation with physicians in specialty,Commitment to best practices,Commitment to quality improvement,Adoption of health information technology,Active fellowship program,Commitment to clinical research,Help for families,Enlists families in structuring care
Hospital,,,,,,,,,,,,,,,,,,,,,,,,,,,,
boston,High,Very high,High,High,Excellent,Below average,Excellent,Very high,Very high,Very high,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,None,Excellent,Excellent,Above average,Excellent,Excellent,Excellent,Excellent,Excellent
chicag,Very high,Very high,Very high,High,Excellent,Below average,Excellent,Average,Average,Average,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Above average,None,Good,Excellent,Above average,Above average,Excellent,Excellent,Excellent,Excellent
cincinnati,High,Very high,High,High,Excellent,Below average,Above average,Average,Very high,Very high,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,None,Very good,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent
la,High,High,Very high,Very high,Excellent,Average,Above average,High,Very high,Very high,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,None,Very good,Excellent,Above average,Excellent,Excellent,Excellent,Excellent,Excellent
stanford,Average,Very high,High,Average,Excellent,Below average,Above average,Very high,Very high,Very high,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,None,Very good,Excellent,Above average,Excellent,Excellent,Excellent,Excellent,Excellent
texas,High,Very high,High,High,Excellent,Average,Excellent,High,Very high,Very high,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,None,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent
umich,High,Very high,High,Very high,Above average,Above average,Above average,Very high,Very high,Very high,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,None,Very good,Excellent,Excellent,Above average,Excellent,Excellent,Excellent,Excellent
wisconsin,Very high,High,Very high,Average,Above average,Excellent,Excellent,High,High,Very high,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,Excellent,None,Good,Excellent,Above average,Excellent,Above average,Excellent,Excellent,Excellent
